# <font color = 'pickle'>**HW - Pre-processing/Feature Engineering - 10 Points**
- **You have to submit two files for this part of the HW (1) ipynb (colab notebook) and (2) pdf file.**
-  **You can use spacy and/or regular expressions for this notebook.**

# <font color ='pickle'> **Task1: Download data and combine data from multiple files into a single dataframe - 2 Points.**

In this task you have to download the moview reviews from the following link:
https://www.cs.cornell.edu/people/pabo/movie-review-data/scale_whole_review.tar.gz
<br>

**Instructions:** <br>

- The data has movie reviews from four different reviewers: 
(1) Dennis+Schwartz,  (2) James+Berardinelli, (3) Scott+Renshaw and (4) Steve+Rhodes.

- You have to extract the reviews of  the four reviewers in a single dataframe.
- The final dataframe should have two columns (1) Moview Review and (2) Reviewer Name.


In [1]:
# Import Libraries
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 830 kB 5.0 MB/s 
     |████████████████████████████████| 281 kB 60.7 MB/s 
     |████████████████████████████████| 1.6 MB 49.0 MB/s 
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16322 sha256=07493aac2deb6d056890905a9986ebed947f30fac945bd7f090e10ca900388ca
  Stored in directory: /root/.cache/pip/wheels/29/a7/0e/3a8f17ac69d759e1e93647114bc9bdc95957e5b0cbfd405205
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [2]:
import pandas as pd
from pathlib import Path
import zipfile
import tarfile
import numpy as np
import swifter
import re
import string
import random
from sklearn.base import BaseEstimator, TransformerMixin
from bs4 import BeautifulSoup
import spacy
import re
import os
from spacy.matcher import Matcher
from spacy.tokens import Token
from pathlib import Path
from nltk.stem.porter import PorterStemmer
from collections import defaultdict
from pathlib import Path

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
basepath = '/content/drive/MyDrive/datasets/'
base_folder = Path(basepath)
archive = base_folder/'archive'
data_folder = base_folder/'data'

Mounted at /content/drive


In [4]:
url = 'https://www.cs.cornell.edu/people/pabo/movie-review-data/scale_whole_review.tar.gz'
!wget {url} -P {archive}

--2022-09-10 23:11:51--  https://www.cs.cornell.edu/people/pabo/movie-review-data/scale_whole_review.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8853204 (8.4M) [application/x-gzip]
Saving to: ‘/content/drive/MyDrive/datasets/archive/scale_whole_review.tar.gz.5’

scale_whole_review. 100%[===================>]   8.44M  26.6MB/s    in 0.3s    

2022-09-10 23:11:51 (26.6 MB/s) - ‘/content/drive/MyDrive/datasets/archive/scale_whole_review.tar.gz.5’ saved [8853204/8853204]



In [ ]:
with  tarfile.open(archive/'scale_whole_review.tar.gz', 'r') as tar:
  tar.extractall(data_folder/'scale_whole_review')

In [5]:

sub_folders = [name for name in os.listdir(data_folder/'scale_whole_review') if os.path.isdir(os.path.join(data_folder/'scale_whole_review', name))]

print(sub_folders)

['Steve+Rhodes', 'Scott+Renshaw', 'James+Berardinelli', 'Dennis+Schwartz']


In [11]:
results = defaultdict(list)
for folder in sub_folders:
  for file in Path(data_folder/'scale_whole_review'/folder/'txt.parag').iterdir():
      with open(file, "r", encoding='ISO-8859-1') as file_open:     
          results["Movie Review"].append(file_open.read())
          results["Reviewer name"].append(folder)
df = pd.DataFrame(results)

In [13]:
df

,Movie Review,Reviewer name
0,Nerdish-looking Theodurus Melville (Hill Harpe...,Steve+Rhodes
1,Is E. Elias Merhige's SHADOW OF THE VAMPIRE a ...,Steve+Rhodes
2,"Sandra Bullock, as FBI field agent Gracie Hart...",Steve+Rhodes
3,So what do you get when you cross two bad movi...,Steve+Rhodes
4,"A 32-year-old painter, Edward (Tonio Arango), ...",Steve+Rhodes
...,...,...
5001,La Religieuse (director: Jacques Rivette; cast...,Dennis+Schwartz
5002,OUT OF SIGHT (director:Steven Soderbergh; cast...,Dennis+Schwartz
5003,A SIMPLE PLAN (director: Sam Raimi; cast: Bill...,Dennis+Schwartz
5004,"MATRIX, THE (directors: Andy Wachowski, Larry ...",Dennis+Schwartz


# <font color ='pickle'> **Task2: Feature Engineering and Preprocessing IMDB - 8 points**


<font color = 'dodgerblue'>**You can use regular expression or spacy for this task** </font>

- <font color = 'dodgerblue'> **PreProcessing**:</font>
  1. Remove HTML tags and new line character (\n)
  2. Remove email, urls and punctuations

  For preprocessing,  write your own simple functions  and your final cleaned text should be saved in a new column - `cleaned_text`.


- <font color = 'dodgerblue'> **Feature Engineering**

  Use the `cleaned_text` column you created in the previous step and extract following features as new column.

  1. number of words
  2. number of characters
  3. number of characters without space
  4. average word length
  5. count of numbers(37, 201, 20 etc.)

You will use the imdb moview review dataset. The details of the data can be found from this link : https://ai.stanford.edu/~amaas/data/sentiment/.

Description of the data from the above link :
"*This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well. Raw text and already processed bag of words formats are provided. See the README file contained in the release for more details.*".

We extracted the data from text files and save  the train and test data as csv files. We further cleaned the dataset. <font size =4, color = 'dodgerblue'>**We will use train.csv file for this task.**</font>.
The file is availibale in Lecture2 data subfolder.

In [ ]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
!wget {url} -P {archive}

--2022-09-08 04:45:44--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘/content/drive/MyDrive/datasets/archive/aclImdb_v1.tar.gz.1’

aclImdb_v1.tar.gz.1 100%[===================>]  80.23M  17.1MB/s    in 12s     

2022-09-08 04:45:56 (6.73 MB/s) - ‘/content/drive/MyDrive/datasets/archive/aclImdb_v1.tar.gz.1’ saved [84125825/84125825]



In [ ]:
with  tarfile.open(archive/'aclImdb_v1.tar.gz', 'r') as tar:
  tar.extractall(data_folder/'aclImdb_v1')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/data/datasets/train.csv',index_col=0)
data.head()

,Reviews,Labels
0,Ever wanted to know just how much Hollywood co...,1
1,The movie itself was ok for the kids. But I go...,1
2,You could stage a version of Charles Dickens' ...,1
3,this was a fantastic episode. i saw a clip fro...,1
4,and laugh out loud funny in many scenes.<br />...,1


In [ ]:
# check the models we have dowloaded in spacy folder
!python -m spacy download en_core_web_sm
!pip install swifter -qq
!pip install -U spacy -qq

2022-09-08 04:46:10.207891: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 12.8 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/data/datasets')

In [ ]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/data/datasets',
 '/content/drive/MyDrive/data/datasets']



PreProcessing:

* Remove HTML tags and new line character (\n)
* Remove email, urls and punctuations

For preprocessing, write your own simple functions and your final cleaned text should be saved in a new column - cleaned_text.




In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
def html_clean(text):
  if (bool(BeautifulSoup(text, "html.parser").find())==True):         
      soup = BeautifulSoup(text, "html.parser")
      text = soup.get_text()
  else:
      pass
  return re.sub(r'[\n]',' ', text) 

In [ ]:
def other_cleans(text):
  matcher = Matcher(nlp.vocab)
  matcher.add("punctuation",[ [{"is_punct": True}]])
  matcher.add("urls", [[{"like_url": True}]])
  matcher.add("emails", [[{"like_email": True}]])
  cleaned_text = []
  Token.set_extension('is_remove', default=False, force=True)
  for doc in nlp.pipe(text):
      matches = matcher(doc)
      for _, start, end in matches:
          for token in doc[start:end]:
              token._.is_remove =True
      text = ' '.join(token.text for token in doc if (token._.is_remove==False))                        
      cleaned_text.append(text)
  return cleaned_text

In [ ]:
# Remove HTML tags and new line character (\n)
x_clean = [html_clean(text) for text in data['Reviews']]


In [ ]:
disabled = nlp.select_pipes(disable= ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer'])

In [ ]:
print(nlp.pipe_names)

['ner']


In [ ]:
#Remove email, urls and punctuations
x_clean_final = other_cleans(x_clean)

In [ ]:
data['cleaned_text'] = x_clean_final

In [ ]:
# Original text
data.iloc[0][0]

'Ever wanted to know just how much Hollywood could get away with before the Hayes Code was officially put into effect? Well, unfortunately "Convention City" is lost, so well just have to watch "Tarzan and His Mate" to find out. For 1934, there is a remarkable amount of sexual innuendo and even exposed flesh. Just look at Jane\'s nude swim. While Tarzan is often thought of as b-adventure films made for young boys and no one else, this picture proves that the series was originally very adult. Over seventy years later, it is still as sexy as it was when it came out.<br /><br />In addition to the envelope pushing taboo nature, it is a superb and exciting adventure story. I\'ve always enjoyed the jungle films that Hollywood churned out in the 30s and the 40s, but there are few from the genre I\'d call great films. "Tarzan and His Mate" is by far the best film from this long gone subgenre. The sequences of the attacks on the safari by either apes or natives still manage to create tension tod

In [ ]:
#Cleaned text
data.iloc[0][2]

"Ever wanted to know just how much Hollywood could get away with before the Hayes Code was officially put into effect Well unfortunately Convention City is lost so well just have to watch Tarzan and His Mate to find out For 1934 there is a remarkable amount of sexual innuendo and even exposed flesh Just look at Jane 's nude swim While Tarzan is often thought of as b adventure films made for young boys and no one else this picture proves that the series was originally very adult Over seventy years later it is still as sexy as it was when it came out In addition to the envelope pushing taboo nature it is a superb and exciting adventure story I 've always enjoyed the jungle films that Hollywood churned out in the 30s and the 40s but there are few from the genre I 'd call great films Tarzan and His Mate is by far the best film from this long gone subgenre The sequences of the attacks on the safari by either apes or natives still manage to create tension today Also the animals are all too c

In [ ]:
# Feature engineering

Use the cleaned_text column you created in the previous step and extract following features as new column.

* number of words
* number of characters
* number of characters without space
* average word length
* count of numbers(37, 201, 20 etc.)

In [ ]:
from numpy.lib.function_base import average
# this cell will take around 10 minutes on colab
word_count = []
char_count = []
char_count_excluding_space = []
ave_word_len =[]
num_count = []
for doc in nlp.pipe(data.cleaned_text.values, batch_size=1000, n_process=-1):
    word = [token.text for token in doc] 
    char_nospace =  [len(token.text) for token in doc ] 
    word_len =  [len(token.text) for token in doc]         
    num_ =  [token.text for token in doc if token.is_digit]

    word_count.append(len(word))
    char_count.append(doc[-1].idx+len(doc[-1])-1)
    char_count_excluding_space.append(sum(char_nospace))
    ave_word_len.append(average(word_len))
    num_count.append(len(num_))

In [ ]:
data['num_of_words'] = word_count
data['num_of_chars'] = char_count
data['num_of_chars_nospace'] = char_count_excluding_space
data['ave_num_of_words'] = ave_word_len
data['count_of_nums'] = num_count
data.head()

,Reviews,Labels,cleaned_text,num_of_words,num_of_chars,num_of_chars_nospace,ave_num_of_words,count_of_nums
0,Ever wanted to know just how much Hollywood co...,1,Ever wanted to know just how much Hollywood co...,243,1290,1050,4.320988,1
1,The movie itself was ok for the kids. But I go...,1,The movie itself was ok for the kids But I got...,59,280,225,3.813559,0
2,You could stage a version of Charles Dickens' ...,1,You could stage a version of Charles Dickens A...,850,4672,3825,4.500000,1
3,this was a fantastic episode. i saw a clip fro...,1,this was a fantastic episode i saw a clip from...,131,635,506,3.862595,0
4,and laugh out loud funny in many scenes.<br />...,1,and laugh out loud funny in many scenes The mo...,105,544,441,4.200000,0
